In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Retinoblastoma"
cohort = "GSE110811"

# Input paths
in_trait_dir = "../../input/GEO/Retinoblastoma"
in_cohort_dir = "../../input/GEO/Retinoblastoma/GSE110811"

# Output paths
out_data_file = "../../output/preprocess/Retinoblastoma/GSE110811.csv"
out_gene_data_file = "../../output/preprocess/Retinoblastoma/gene_data/GSE110811.csv"
out_clinical_data_file = "../../output/preprocess/Retinoblastoma/clinical_data/GSE110811.csv"
json_path = "../../output/preprocess/Retinoblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains microarray data (gene expression profiles)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Retinoblastoma): The tissue is already identified in row 0, but we're interested in anaplasia grade
trait_row = 1  # anaplasia grade is our trait of interest
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # If value contains a colon, extract the part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert anaplasia grades to binary (Severe vs Others)
    if 'severe' in value.lower():
        return 1  # Severe anaplasia
    elif 'mild' in value.lower() or 'moderate' in value.lower():
        return 0  # Mild or Moderate anaplasia
    else:
        return None  # For normal retina or retinocytoma or unknown values

# No age data to convert
def convert_age(value):
    return None

# No gender data to convert
def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering based on trait and gene availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to read the sample characteristics data correctly
    # Let's load the GEO matrix file and extract sample characteristics
    import gzip
    
    # Function to parse the GEO matrix file for sample characteristics
    def extract_sample_characteristics(matrix_file):
        sample_chars = {}
        samples = []
        
        with gzip.open(matrix_file, 'rt') as f:
            line_idx = 0
            in_sample_section = False
            
            for line in f:
                line = line.strip()
                
                # Find the sample section
                if line.startswith('!Sample_geo_accession'):
                    in_sample_section = True
                    samples = line.split('\t')[1:]
                    continue
                
                # Process sample characteristics
                if in_sample_section and line.startswith('!Sample_characteristics_ch1'):
                    values = line.split('\t')[1:]
                    unique_values = list(set(values))
                    sample_chars[line_idx] = unique_values
                    line_idx += 1
                
                # End of sample section
                if in_sample_section and line.startswith('!Sample_data_row_count'):
                    break
        
        # Create a DataFrame for sample characteristics
        clinical_df = pd.DataFrame()
        for idx, samples_values in sample_chars.items():
            for val in samples_values:
                if pd.isna(val):
                    continue
                characteristic = val.split(':', 1)[0].strip() if ':' in val else val
                clinical_df.loc[idx, 'characteristic'] = characteristic
                clinical_df.loc[idx, 'values'] = [val for val in samples_values if not pd.isna(val)]
        
        return clinical_df, samples, sample_chars
    
    # Extract sample characteristics from the matrix file
    matrix_file = os.path.join(in_cohort_dir, "GSE110811_series_matrix.txt.gz")
    clinical_df, samples, sample_chars = extract_sample_characteristics(matrix_file)
    
    # Create full clinical data DataFrame for all samples
    full_clinical_data = pd.DataFrame(index=samples)
    
    # Process sample characteristics into proper format for geo_select_clinical_features
    for idx, values in sample_chars.items():
        sample_values = {}
        for i, sample in enumerate(samples):
            value_idx = min(i, len(values) - 1) if i < len(values) else 0
            sample_values[sample] = values[value_idx] if value_idx < len(values) else None
        
        full_clinical_data[idx] = pd.Series(sample_values)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=full_clinical_data,
        trait="Anaplasia",  # Using a more specific trait name based on the data
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement a clean, complete solution for Step 3:

```python
# First, let's look at the GSE data and series matrix file to understand the dataset
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Callable, Optional

# Let's check the directory structure and files
files = os.listdir(in_cohort_dir)
print(f"Files in {in_cohort_dir}:")
for file in files:
    print(f"  {file}")

# Load the series matrix file if it exists (it should be named with a "series_matrix" in the filename)
series_matrix_files = [f for f in files if "series_matrix" in f.lower()]

if series_matrix_files:
    series_matrix_file = os.path.join(in_cohort_dir, series_matrix_files[0])
    
    # Load the data using pandas read_csv, which can handle most text-based formats
    try:
        # We need to get sample characteristics which are typically in the file header
        sample_chars = {}
        with open(series_matrix_file, 'r') as f:
            reading_chars = False
            char_index = 0
            for line in f:
                if line.startswith('!Sample_characteristics_ch1'):
                    reading_chars = True
                    sample_chars[char_index] = line.strip().split('\t')[1:]
                    char_index += 1
                elif reading_chars and not line.startswith('!Sample_characteristics_ch1'):
                    reading_chars = False
                
                # Also check for gene expression data indicator
                if "!dataset_platform = " in line:
                    platform = line.strip().split('= ')[1]
                    print(f"Platform: {platform}")
                
                # Get background information
                if "!Series_summary" in line:
                    summary = line.strip().split('= ')[1]
                    print(f"Series summary: {summary}")
                
                # Look for gene expression data indication
                if "!Series_type = " in line:
                    series_type = line.strip().split('= ')[1]
                    print(f"Series type: {series_type}")
                
                # Sample info
                if "!Sample_title" in line:
                    sample_titles = line.strip().split('\t')[1:]
                    print(f"Sample titles: {sample_titles}")
        
        # Print sample characteristics for analysis
        print("\nSample characteristics:")
        for idx, chars in sample_chars.items():
            unique_values = set(chars)
            print(f"Row {idx}: {list(unique_values)[:5]} {'...' if len(unique_values) > 5 else ''} (Unique values: {len(unique_values)})")

    except Exception as e:
        print(f"Error reading series matrix file: {e}")
else:
    print("No series matrix file found in the directory.")

# Now let's determine gene expression availability
# Based on platform info - assuming gene expression data is available unless proven otherwise
is_gene_available = True  # Will be updated based on platform/series_type if needed

# For clinical data extraction, we'll create conversion functions
def convert_trait(value):
    """Convert trait value to binary (0 for non-case, 1 for case)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Retinoblastoma specific conversion
    value = value.lower()
    if 'retinoblastoma' in value or 'rb' in value or 'tumor' in value or 'cancer' in value:
        return 1
    elif 'normal' in value or 'control' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to numeric (continuous)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numeric age
    try:
        # Look for patterns like "X years" or "X months"
        if 'year' in value.lower():
            age = float(value.lower().split('year')[0].strip())
            return age
        elif 'month' in value.lower():
            age = float(value.lower().split('month')[0].strip()) / 12
            return age
        elif 'day' in value.lower():
            age = float(value.lower().split('day')[0].strip()) / 365
            return age
        else:
            # Try direct conversion
            return float(value.strip())
    except:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# Based on the output analysis, we'll determine the row indices and data availability
trait_row = None
age_row = None
gender_row = None

# After analyzing the output, update the variable assignments and continue with processing
if 'sample_chars' in locals() and sample_chars:
    # Convert sample_chars to DataFrame for processing
    clinical_df = pd.DataFrame(sample_chars).T
    
    # Analyze each row in sample_chars to find trait, age, and gender information
    for idx, values in sample_chars.items():
        unique_values = list(set(values))
        values_str = ' '.join([str(v).lower() for v in unique_values])
        
        # Check for trait information
        if any('retinoblastoma' in str(v).lower() or 'rb' in str(v).lower() or 
               'tumor' in str(v).lower() or 'normal' in str(v).lower() or 
               'control' in str(v).lower() for v in unique_values):
            trait_row = idx
        
        # Check for age information
        if any('age' in str(v).lower() or 'year' in str(v).lower() or 
               'month' in str(v).lower() for v in unique_values):
            age_row = idx
        
        # Check for gender information
        if any('gender' in str(v).lower() or 'sex' in str(v).lower() or 
               'male' in str(v).lower() or 'female' in str(v).lower() 
               for v in unique_values):
            gender_row = idx
    
    # Determine trait data availability
    is_trait_available = trait_row is not None
    
    # Save the initial validation information
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
    
    # If trait data is available, extract clinical features
    if is_trait_available:
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the clinical data to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df


### Step 4: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# These identifiers are numerical and don't match the pattern of human gene symbols
# Human gene symbols typically have a format like "BRCA1", "TP53", etc.
# These appear to be probe IDs or feature IDs from a microarray platform that need mapping

# Based on my biomedical knowledge, these are not standard human gene symbols
# They appear to be probe identifiers (likely Affymetrix or another platform)
# We will need to map these identifiers to proper gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


### Step 7: Gene Identifier Mapping

In [ ]:
# First, re-extract the gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file_path)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# 1. Identify which columns to use from the gene annotation data
print("Checking annotation columns for mapping information...")

# Display all column names to find potential gene symbol columns
print("All available columns in gene annotation:")
print(list(gene_annotation.columns))

# Check a sample of rows to see what other columns might contain gene information
print("\nSample row from annotation data:")
sample_row = gene_annotation.iloc[100].to_dict()
for key, value in sample_row.items():
    print(f"{key}: {value}")

# Let's try to find columns that might contain gene symbols or IDs
potential_gene_cols = []
for col in gene_annotation.columns:
    # Look for column names that might suggest gene information
    if any(term in col.lower() for term in ['gene', 'symbol', 'name', 'title', 'description']):
        non_nan = gene_annotation[col].notna().sum()
        total = len(gene_annotation)
        print(f"Column {col}: {non_nan}/{total} non-null values ({non_nan/total:.2%})")
        potential_gene_cols.append(col)

# Function to extract gene names from location or accession information
def extract_gene_info(row):
    """Attempt to extract gene information from various fields"""
    # Try GB_ACC first as it's a direct link to gene identifiers
    if pd.notna(row.get('GB_ACC')):
        return row['GB_ACC']
    
    # Return None if no gene info found
    return None

# Add a Gene column to our annotation by combining information from multiple sources
gene_annotation['Gene'] = gene_annotation.apply(extract_gene_info, axis=1)

# How many genes did we successfully identify?
gene_count = gene_annotation['Gene'].notna().sum()
print(f"\nIdentified {gene_count} genes out of {len(gene_annotation)} rows ({gene_count/len(gene_annotation):.2%})")

# 2. Get gene mapping dataframe using the ID column and our newly created Gene column
prob_col = 'ID'
gene_col = 'Gene'

# Filter out rows without gene information
gene_annotation_filtered = gene_annotation.dropna(subset=[gene_col])
print(f"After filtering, {len(gene_annotation_filtered)} rows with gene information remain")

mapping_df = gene_annotation_filtered[[prob_col, gene_col]].copy()
mapping_df = mapping_df.astype({prob_col: 'str'})

# Print mapping information
print(f"Generated mapping for {len(mapping_df)} probes")
print("First few rows of mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression data
print(f"\nMapped gene expression data shape: {gene_data.shape}")
if not gene_data.empty:
    print("First few gene symbols:")
    print(gene_data.index[:10])
    
    # Let's check for rows with NaN values
    nan_count = gene_data.isna().sum().sum()
    print(f"Total NaN values in gene expression data: {nan_count}")
else:
    print("WARNING: Gene mapping resulted in empty dataframe")
    
    # If gene mapping failed, let's try direct comparison between IDs
    # This is a fallback approach in case the annotation doesn't provide good mapping
    common_ids = set(gene_data.index) & set(mapping_df['ID'])
    print(f"Number of probe IDs in common between expression data and mapping: {len(common_ids)}")
    
    # If the dataset really doesn't have good mapping, we might need to use the probe IDs directly
    if len(common_ids) > 0:
        print("Some probe IDs match between expression and annotation data")
    else:
        print("No matching probe IDs found between expression and annotation data")


### Step 8: Gene Identifier Mapping

In [ ]:
# First, get the file paths again to ensure they're defined
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# 1. Load gene expression data 
gene_data = get_genetic_data(matrix_file_path)
print(f"Gene expression data shape: {gene_data.shape}")

# 2. Extract gene annotation data
gene_annotation = get_gene_annotation(soft_file_path)
print(f"Gene annotation data shape: {gene_annotation.shape}")

# 3. Examine the gene annotation data and determine which columns to use for mapping
print("Column names in gene annotation:")
print(list(gene_annotation.columns))

# Compare example IDs from gene_data and annotation to check for format inconsistencies
print("\nSample probe IDs from gene_data:")
print(list(gene_data.index[:5]))
print("\nSample probe IDs from gene_annotation:")
print(gene_annotation['ID'].head(5).tolist())

# Based on the gene annotation preview from step 6, we can see that GB_ACC contains RefSeq accessions, 
# not human gene symbols. Let's look for a more suitable column or create a custom mapping.

# Let's check if there are any columns with gene symbols in the entire annotation dataframe
for col in gene_annotation.columns:
    sample_values = gene_annotation[col].dropna().head(10).tolist()
    print(f"\nSample values from column '{col}':")
    print(sample_values)

# Based on inspection, we can see that the 'GB_ACC' column contains gene identifiers (RefSeq) but not human gene symbols
# The 'SPOT_ID' column might contain location information we can use
# Let's extract gene IDs from these columns and then see if we can proceed with mapping

# Create a custom mapping combining available information
mapping_df = gene_annotation[['ID']].copy()
mapping_df = mapping_df[mapping_df['ID'].isin(gene_data.index)]  # Only keep rows with IDs that match gene_data
print(f"\nMatching probe IDs found: {len(mapping_df)}")

if len(mapping_df) > 0:
    # Add GB_ACC as a Gene column when available
    mapping_df['Gene'] = gene_annotation.loc[mapping_df.index, 'GB_ACC']
    
    # Apply gene mapping to convert probe-level data to gene-level data
    print("Converting probe-level measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Print information about the resulting gene expression data
    print(f"Gene expression data shape after mapping: {gene_data.shape}")
    
    if gene_data.empty:
        print("WARNING: Gene mapping resulted in empty dataframe. Using probe IDs directly.")
        # Fall back to using the original gene_data
        gene_data = get_genetic_data(matrix_file_path)
        print(f"Using original gene expression data: {gene_data.shape}")
    else:
        print("First few gene symbols after mapping:")
        print(gene_data.index[:10])
else:
    print("WARNING: No matching probe IDs found. Using original gene expression data.")
    # Keep the original gene_data
    gene_data = get_genetic_data(matrix_file_path)
    print(f"Using original gene expression data: {gene_data.shape}")

# Save the gene expression data for later use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since our dataset lacks clinical features (trait_row=None as determined in Step 2),
# we need a different approach for linking data
# Create a minimal clinical DataFrame with just sample IDs
sample_ids = normalized_gene_data.columns
clinical_features = pd.DataFrame(index=sample_ids)

# Add placeholder for trait column (all NaN)
clinical_features[trait] = float('nan')

# 3 & 4. Since we don't have trait data, we can't properly handle missing values
# or evaluate whether the trait is biased. Set appropriate flags.
is_trait_biased = True  # No trait data means we can't use this cohort for association studies
print("No trait data available for this cohort, marking as biased.")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=False,  # We determined earlier that trait data is not available
    is_biased=is_trait_biased, 
    df=clinical_features,
    note="Dataset contains gene expression data from ovarian cancer cell lines but lacks Retinoblastoma classification information."
)

# 6. We've determined the data is not usable for association studies due to lack of trait information
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # This block likely won't execute but included for completeness
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # We don't have useful linked data to save
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset lacks trait information needed for association studies.")